In [ ]:
import wellmap
import pandas as pd
import os
import string
import seaborn as sns
from os import listdir
from os.path import isfile, join

Old one for single images

In [ ]:
path_to_data = r"C:\Users\Modern\Documents\Biorad\CellProfiler\Results\2024-07-19 RD, Vero, hTERT CK-122\ExpWells_plate.csv"
path_to_wellmap = r"C:\Users\Modern\Documents\Biorad\CellProfiler\Results\2024-07-19 RD, Vero, hTERT CK-122\2024-07-19 RD CK-122 5H (Multichannel).toml"

data_raw = pd.read_csv(path_to_data) 

wellmap.show(path_to_wellmap)

print(data_raw.columns)
# print(data)


In [ ]:
part_to_folder = r"C:\Users\Modern\Documents\Biorad\CellProfiler\Results\2024-07-19 RD, Vero, hTERT CK-122"

# files = [f for f in listdir(part_to_folder) if isfile(join(part_to_folder, f))]

image_list_path = os.path.join(part_to_folder, "ExpImage.csv")
data_path  = os.path.join(part_to_folder, "ExpWells_plate.csv")

data_raw_df = pd.read_csv(data_path) 
image_list_df = pd.read_csv(image_list_path) 

print(image_list_df)


Generating labels for wells

In [ ]:
# row_column_df = pd.DataFrame(columns=['row','col'])

row =  list(string.ascii_uppercase[0:8])
column = list(range(1,13))
plate = image_list_df['FileName_RGB'].to_list()

row_column_list = []


for p in plate:
    for c in column:
        for r in row:
            array_i = [r,c,p]
            row_column_list.append(array_i)

row_column_df = pd.DataFrame(row_column_list, columns = ['row', 'col','FileName_RGB']) 

print(row_column_df)

In [ ]:
data = data_raw[['Intensity_IntegratedIntensity_OrigGreen','Intensity_IntegratedIntensity_OrigRed','Intensity_MeanIntensity_OrigGreen','Intensity_MeanIntensity_OrigRed']].copy()
data_indexed = pd.concat([row_column_df, data], axis=1, join='inner')


data_indexed['col']=data_indexed['col'].astype(str)


Add images names to data (thank you CellProfiler for convenience)

In [26]:
data_full_df =  pd.merge(data_raw_df, image_list_df, on='ImageNumber')
data_indexed = pd.concat([row_column_df, data_full_df], axis=1, join='inner')

print(data_indexed)

    row  col                                   FileName_RGB  ImageNumber  \
0     A    1     2024-07-19 RD CK-122 5H (Multichannel).tif            1   
1     B    1     2024-07-19 RD CK-122 5H (Multichannel).tif            1   
2     C    1     2024-07-19 RD CK-122 5H (Multichannel).tif            1   
3     D    1     2024-07-19 RD CK-122 5H (Multichannel).tif            1   
4     E    1     2024-07-19 RD CK-122 5H (Multichannel).tif            1   
..   ..  ...                                            ...          ...   
283   D   12  2024-07-19 hTERT CK-122 5H (Multichannel).tif            3   
284   E   12  2024-07-19 hTERT CK-122 5H (Multichannel).tif            3   
285   F   12  2024-07-19 hTERT CK-122 5H (Multichannel).tif            3   
286   G   12  2024-07-19 hTERT CK-122 5H (Multichannel).tif            3   
287   H   12  2024-07-19 hTERT CK-122 5H (Multichannel).tif            3   

     ObjectNumber  Intensity_IntegratedIntensity_OrigGreen  \
0               1        

In [ ]:
labels = wellmap.load(path_to_wellmap)
# print(labels.dtypes)
# print(data_indexed.dtypes)


data_plate = pd.merge(data_indexed, labels)
# print(labels.dtypes)

data_plate[["conc_uM"]] = data_plate[["conc_uM"]].apply(pd.to_numeric)

print(data_plate.dtypes)


Drop blank and outliers

In [ ]:
data_plate_wo_blank = data_plate[data_plate.blank == 'No'].copy()

well_drop_list = ['G12']

# data_plate_wo_blank = data_plate_wo_blank[~data_plate_wo_blank['well'].isin(well_drop_list)]

# df_show = data_plate_wo_blank.loc[(data_plate_wo_blank['sample'] == 'Substance_1') & (data_plate_wo_blank['conc_uM'] == '0')]

df_show = data_plate_wo_blank.loc[(data_plate_wo_blank['conc_uM'] == 0)]

print(df_show[['Intensity_MeanIntensity_OrigGreen','well']])

Relative values calculation


In [ ]:
blank_green = data_plate[(data_plate['blank'] == 'Yes')].loc[:,'Intensity_MeanIntensity_OrigGreen'].mean()

print(blank_green)

control_green = data_plate_wo_blank[(data_plate_wo_blank['conc_uM'] == 0)].loc[:,'Intensity_MeanIntensity_OrigGreen'].mean()

print(control_green)



data_plate_wo_blank['Green'] = data_plate_wo_blank['Intensity_MeanIntensity_OrigGreen'] - blank_green

data_plate_wo_blank['Green_relative'] = data_plate_wo_blank['Green']/control_green


df_show = data_plate_wo_blank.loc[(data_plate_wo_blank['conc_uM'] == 0)]

print(df_show[['Green_relative','well']])


Plotting


In [ ]:
# sns.swarmplot(data=data_plate_wo_blank, x="conc_uM", y="Intensity_MeanIntensity_OrigGreen", hue='sample')
# ax = sns.scatterplot(data=data_plate_wo_blank, x="conc_uM", y="Intensity_MeanIntensity_OrigGreen")

ax = sns.relplot(data=data_plate_wo_blank, x="conc_uM", y="Intensity_MeanIntensity_OrigGreen", kind="line", hue = 'cells')

# ax.set(xlabel=None, ylabel='Viable cell, % of control')

ax.set(xlabel='Concentration, μM', ylabel='Green channel intensity')
ax.set(xscale="log")


# handles, labels = ax.get_legend_handles_labels()
# labels = ['CK-122','ISK-1','Control']
# ax.legend(handles=handles, labels=labels)

In [ ]:
ax = sns.swarmplot(data=data_plate_wo_blank, x="conc_uM", y="Intensity_MeanIntensity_OrigGreen")
